In [77]:
# basic test
import numpy as np
from scipy.spatial import distance

def cal_cosine_distance(x, y):
    dist = distance.cosine(x, y)
    return dist

a = np.array([1, 1, 2, 3, 4])
b = np.array([1, 2, 2, 3, 4])

c = a.dot(b)/(np.linalg.norm(a) * np.linalg.norm(b)) # higher -> more similar
d = cal_cosine_distance(a, b) # smaller -> more similar

print(c)
print(d)

0.985665765813916
0.014334234186084038


# prepared functions

In [255]:

import os
import pandas as pd
import copy
import numpy as np
from scipy.spatial import distance
from PIL import Image
import numpy as np

# 
def cal_cosine_distance(x, y):
    # The smaller value, the more similar for x and y
    dist = distance.cosine(x, y)
    return dist

def cal_cosine_similarity(x, y):
    # The bigger value, the more similar for x and y
    sim = 1 - distance.cosine(x, y)
    return sim

def get_img_array(path):
    tmp_img = Image.open(path)
    # reshape to (320, 240)
    tmp_img_reshape = tmp_img.resize((320, 240))
    
    # convert the images to (R,G,B) arrays
    tmp_img_array = np.array(tmp_img_reshape)
    
    # flatten the arrays so they are 1 dimensional vectors
    tmp_img_array = tmp_img_array.flatten()
    
    # divide the arrays by 255, the maximum RGB value to make sure every value is on a 0-1 scale
    tmp_img_array = tmp_img_array/255
    
    return tmp_img_array

# rescale the distance value
def rescale(result):
    test = copy.deepcopy(result)
    sorted_test = sorted(test.items(), key=lambda item: item[1][2], reverse = True)
    print('max is:', sorted_test[0][1][2], '\nmin is:', sorted_test[-1][1][2])
    max_v = sorted_test[0][1][2]
    min_v = sorted_test[-1][1][2]
    print('rescaling...')
#     print(len(sorted_test))
    for i in range(len(sorted_test)):
        sorted_test[i][1][2] = (sorted_test[i][1][2] - min_v) / (max_v - min_v)
    print('rescaled.')
    return sorted_test

def get_target(emotion_name):
    # Anger, Disgust, Fear, Happiness, Sadness, Surprise, Neutral
    # or lowercase
    if emotion_name in ["Anger", "anger"]:
        return 0
    elif emotion_name in ["Disgust", "disgust"]:
        return 1
    elif emotion_name in ["Fear", "fear"]:
        return 2
    elif emotion_name in ["Happiness", "happiness"]:
        return 3
    elif emotion_name in ["Sadness", "sadness"]:
        return 4
    elif emotion_name in ["Surprise", "surprise"]:
        return 5
    elif emotion_name in ["Neutral", "neutral"]:
        return 6


# main loop for dealing with cosine distance 

In [257]:

COPY_FLAG = 1 # 1 is copy, 0 - no copy
NUM_OF_SELECT = 100

# get all img array, for anger
for target_emotion in ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']:
    folderName = os.path.join(target_emotion)
    ls_name = os.listdir(folderName)
    result_dic = {}

    for i in ls_name:
        if 'emotion' in i:
            temp = pd.read_csv(os.path.join(folderName, i))
            result = temp[target_emotion][0]
            result = round(result, 6)
            fileName = temp['input'][0]
            result_dic[i[:-12] + '.png'] = result
    # sort in value order 
    res_value_order = sorted(result_dic.items(), key=lambda item: item[1], reverse = True)

    # get all img array
    res_img_array = {}

    for i in res_value_order:
        temp_img_array = get_img_array(os.path.join(target_emotion, i[0]))
        res_img_array[i] = [i[1], temp_img_array]

    # compute all similarity
    data_length = len(res_img_array.keys())
    nameList = list(res_img_array.keys())
    cos_dis_result = {}
    cos_dis_result_listform = []
    counter = 0
    for i in range(data_length):
        for j in range(i + 1, data_length):
            vec1 = res_img_array[nameList[i]][1]
            vec2 = res_img_array[nameList[j]][1]
            cos_dis = cal_cosine_distance(vec1, vec2)
            if cos_dis == 0.0:
                raise Exception('ERROR, counter is ' + str(counter), i, j)
            tmp = [nameList[i], nameList[j], cos_dis]
            cos_dis_result_listform.append(tmp)
            cos_dis_result[counter] = tmp
            counter += 1
    rescale_cos_dis_result = rescale(cos_dis_result)

    # in this dic, I use all img name as the first id of the dic, there inner dic, which contains all the img name except itself.
    # I store the rating in the inner dic
    res_dic_rescaled_cos_dis = {i:{} for i in res_img_array.keys()}

    for i in rescale_cos_dis_result:
        tempa = i[1][0]
        tempb = i[1][1]
        temp_score = i[1][2]
        res_dic_rescaled_cos_dis[tempa][tempb] = temp_score
        res_dic_rescaled_cos_dis[tempb][tempa] = temp_score    

    # greedy search

    count = 1
    finalResult = []
    finalResult.append(res_value_order[0])
    finalResult.append(res_value_order[-1])
    x_list = []
    # print(finalResult)
    for i in range(len(finalResult)):
        x_list.append(finalResult[i][0])
    # print(x_list)
    for count in range(NUM_OF_SELECT):
        # for storing the distance from the selected list to others 
        y_list = []

        # find intersect keys, which is not in x_list
        cur_keys = ''

        for i in x_list:
            if cur_keys == '':
                cur_keys = res_dic_rescaled_cos_dis[i].keys()
            else:
                type(cur_keys)
                cur_keys = cur_keys & res_dic_rescaled_cos_dis[i].keys()
        cur_dic = dict.fromkeys(cur_keys, 0)
        for k in x_list:
            assert k not in cur_dic 
        for i in cur_dic.keys():
            for j in x_list:
                cur_dic[i] += res_dic_rescaled_cos_dis[i][j]
    #     print(len(cur_keys))
        largeest_dis_target_name = max(cur_dic, key=cur_dic.get)
        pos_of_largeest_dis_target_name = [index1 for index1,value1 in 
         enumerate(res_value_order) for index2,value2 in enumerate(value1) if value2==largeest_dis_target_name][0]
        x_list.append(largeest_dis_target_name)
        finalResult.append(res_value_order[pos_of_largeest_dis_target_name])


    # Copy files
    sum_folderName = os.path.join(target_emotion, 'selected{}'.format(NUM_OF_SELECT))
    img_only_folderName = os.path.join(target_emotion, 'selected_imgonly{}'.format(NUM_OF_SELECT))
    if COPY_FLAG and not os.path.exists(sum_folderName):
        try:
            os.mkdir(sum_folderName)
        except Exception as e:
            print(e)
    if COPY_FLAG and not os.path.exists(img_only_folderName):
        try:
            os.mkdir(img_only_folderName)
        except Exception as e:
            print(e)

    import shutil
    for i in finalResult:
        # i is fig name
        print(i[0])
        img_file_name = os.path.join(target_emotion, i[0])
        csv_file_name = os.path.join(target_emotion, i[0][:-4] + '.csv')
        axes_file_name = os.path.join(target_emotion, i[0][:-4] + '_axes_data.csv')
        emotion_file_name = os.path.join(target_emotion, i[0][:-4] + '_emotion.csv')

        # all files
        if COPY_FLAG and not os.path.exists(os.path.join(sum_folderName, i[0])):
            shutil.copy(img_file_name, sum_folderName)
            print('copied0')
        if COPY_FLAG and not os.path.exists(os.path.join(sum_folderName, i[0][:-4] + '.csv')):
            shutil.copy(csv_file_name, sum_folderName)
            print('copied1')
        if COPY_FLAG and not os.path.exists(os.path.join(sum_folderName, i[0][:-4] + '_axes_data.csv')):
            shutil.copy(axes_file_name, sum_folderName)
            print('copied2')
        if COPY_FLAG and not os.path.exists(os.path.join(sum_folderName, i[0][:-4] + '_emotion.csv')):
            shutil.copy(emotion_file_name, sum_folderName)
            print('copied3')

        if COPY_FLAG and not os.path.exists(os.path.join(img_only_folderName, i[0])):
            shutil.copy(img_file_name, img_only_folderName)
            print('copied4')

SyntaxError: invalid syntax (2981895799.py, line 99)

'2023_02_10_15_12_04_anger_8.png'

In [86]:
print(rescale_cos_dis_result[0])
print(rescale_cos_dis_result[-1])

(55302, ['2023_02_10_15_30_46_anger_490.png', '2023_02_10_15_12_04_anger_8.png', 1.0])
(1509, ['2023_02_10_15_24_05_anger_335.png', '2023_02_10_15_24_08_anger_336.png', 0.0])


[[0, ('2023_02_10_15_28_04_anger_433.png', 0.890863)],
 [0, ('2023_02_10_15_25_54_anger_381.png', 0.000222)]]

In [139]:
cos_dis_result_listform[0]
cos_dis_result_listform[1]

[('2023_02_10_15_28_04_anger_433.png', 0.890863),
 ('2023_02_10_15_28_40_anger_446.png', 0.879018),
 6.64947043207409e-05]

In [137]:
# construct a 500 x 500 matrix
mat_2d = []
print(len(cos_dis_result_listform))
for i in range(500*500):
    mat_2d.append(cos_dis_result_listform[i][2])
ary = np.array(mat_2d).reshape(500, 500)

124750


IndexError: list index out of range

In [ ]:

print('Final selection for ' + target_emotion, finalResult)

# show the result
finalResult = sorted(finalResult, key=lambda x:x[0])
print('Final selection for ' + target_emotion, finalResult)
# Copy files
sum_folderName = "image_analysis/211223Exp/Exp7/selected_files/" + target_emotion + '/'
if not os.path.exists(sum_folderName):
    try:
        os.mkdir(sum_folderName)
    except Exception as e:
        print(e)
import shutil
for i in finalResult:
    temp = pd.read_csv(folderName + i[1][:-4] + '_emotion.csv')
    targetID = RCSystem.get_target(target_emotion)
    temp_result = temp.iloc[:, -8:].iloc[0, targetID]
    print(i[0], i[1][:-4]+'.png', temp_result)
    tempFileAU = i[1]
    tempFileFig = i[1][:-4]+'.png'
    tempFilePyfeat = i[1][:-4] + '_emotion.csv'
#         shutil.copy(folderNameAU + tempFileAU, sum_folderName)
#         shutil.copy(folderName + tempFileFig, sum_folderName)
#         shutil.copy(folderName + tempFilePyfeat, sum_folderName)

max is: 0.003410925908368223 
min is: 6.420692183173315e-05
rescaling...
rescaled.


In [58]:
# for i in range(5):
#     print(test_rescle[i])

# for i in range(-5, 0):
#     print(test_rescle[i])

In [ ]:
# greedy search

count = 1
#     print('top_name_list', top_name_list, '\n')
finalResult = []
finalResult.append([0, res_value_order[0]])

# construct a 10 x 10 matrix
mat_2d = []
for i in range(100):
    mat_2d.append(result_AU2[i][2])
ary = np.array(mat_2d).reshape(10, 10)

#     finalResult.append([2, top_name_list[2]])
while count < 5:
    x_list = []
    y_list = []
#         print()
    for i in range(len(finalResult)):
        x_list.append(finalResult[i][0])

    for i in x_list:
        y_list.append(ary[i])
#         print(y_list)
#     print(ary[i])
#     print(np.argmax(ary[i]))
    sum_v = np.zeros(10)
    for i in y_list:
        sum_v += i
    for i in x_list:
        sum_v[i] = 0
    y = np.argmax(sum_v)
    finalResult.append([y, top_name_list[y]])
    count += 1
    print('x_list', x_list)
#         print(sum_v, y)
#         np.partition(k.flatten(), -2)[-2]
#         print(ary[x].index(max(ary[x])))
#     print(max(ary[x]))
print('Final selection for ' + target_emotion, finalResult)

# show the result
finalResult = sorted(finalResult, key=lambda x:x[0])
print('Final selection for ' + target_emotion, finalResult)

### For Test

In [82]:
# For test
    
    
# open images
folderName = 'anger'
a = os.path.join(folderName, '2023_02_10_15_11_49_anger_0.png')
b = os.path.join(folderName, '2023_02_10_15_11_53_anger_2.png')
# for i in range(50):
# a = os.path.join(folderName, res_value_order[0][0][:-12]+'.png')
# b = os.path.join(folderName, res_value_order[i][0][:-12]+'.png')
img1 = Image.open(a)
img2 = Image.open(b)

# make sure images have same dimensions, use .resize to scale image 2 to match image 1 dimensions
# also reduce the shape by half just to save some processing power

img1_reshape = img1.resize((round(img1.size[0]*0.5), round(img1.size[1]*0.5)))
img2_reshape = img2.resize((round(img1.size[0]*0.5), round(img1.size[1]*0.5)))
# img1 (640, 480)
# img2 (640, 480)
# img1_reshape (320, 240)

# convert the images to (R,G,B) arrays
img_array1 = np.array(img1_reshape)
img_array2 = np.array(img2_reshape)

# flatten the arrays so they are 1 dimensional vectors
img_array1 = img_array1.flatten()
img_array2 = img_array2.flatten()

# divide the arrays by 255, the maximum RGB value to make sure every value is on a 0-1 scale
# img_array1 = img_array1/255
# img_array2 = img_array2/255

print(cal_cosine_distance(img_array1, img_array2))
print(cal_cosine_similarity(img_array1, img_array2))
print(a, b)

0.11000954841219102
0.889990451587809
anger/2023_02_10_15_11_49_anger_0.png anger/2023_02_10_15_11_53_anger_2.png


In [ ]:
# greedy search

    count = 1
#     print('top_name_list', top_name_list, '\n')
    finalResult = []
    finalResult.append([0, top_name_list[0]])
    
    # construct a 10 x 10 matrix
    mat_2d = []
    for i in range(100):
        mat_2d.append(result_AU2[i][2])
    ary = np.array(mat_2d).reshape(10, 10)
    
#     finalResult.append([2, top_name_list[2]])
    while count < 5:
        x_list = []
        y_list = []
#         print()
        for i in range(len(finalResult)):
            x_list.append(finalResult[i][0])

        for i in x_list:
            y_list.append(ary[i])
#         xprint(y_list)
    #     print(ary[i])
    #     print(np.argmax(ary[i]))
        sum_v = np.zeros(10)
        for i in y_list:
            sum_v += i
        for i in x_list:
            sum_v[i] = 0
        y = np.argmax(sum_v)
        finalResult.append([y, top_name_list[y]])
        count += 1
        print('x_list', x_list)
#         print(sum_v, y)
    #         np.partition(k.flatten(), -2)[-2]
    #         print(ary[x].index(max(ary[x])))
    #     print(max(ary[x]))
    print('Final selection for ' + target_emotion, finalResult)
    
    # show the result
    finalResult = sorted(finalResult, key=lambda x:x[0])
    print('Final selection for ' + target_emotion, finalResult)
    
    
    # Copy files
    sum_folderName = "image_analysis/211223Exp/Exp7/selected_files/" + target_emotion + '/'
    if not os.path.exists(sum_folderName):
        try:
            os.mkdir(sum_folderName)
        except Exception as e:
            print(e)
    import shutil
    for i in finalResult:
        temp = pd.read_csv(folderName + i[1][:-4] + '_emotion.csv')
        targetID = RCSystem.get_target(target_emotion)
        temp_result = temp.iloc[:, -8:].iloc[0, targetID]
        print(i[0], i[1][:-4]+'.png', temp_result)
        tempFileAU = i[1]
        tempFileFig = i[1][:-4]+'.png'
        tempFilePyfeat = i[1][:-4] + '_emotion.csv'
#         shutil.copy(folderNameAU + tempFileAU, sum_folderName)
#         shutil.copy(folderName + tempFileFig, sum_folderName)
#         shutil.copy(folderName + tempFilePyfeat, sum_folderName)
    